## Task 9
### Solving Heat equation by finite difference method (implicit and explicit schemes)

$ u_{t}(x, t) = \kappa u_{xx}(x, t) + f(x, t), $ <br/>
$ \kappa = const > 0, 0 < x < a, 0 < t \le T $ <br/>
Initial condition: <br/>
$ u(x, 0) = \mu(x), 0 \le x \le a $ <br/> 
Boundary condition: <br/>
$ u(0, t) = \mu_{1}(t), u(a, t) = \mu_{2}(t), 0 \le t \le T $  <br/>

In [11]:
import numpy as np
from typing import Tuple, Callable, Literal
from math import cos
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.sparse import lil_matrix, csc_matrix
from scipy.sparse.linalg import spsolve

Function = Callable[[float], float]
BinaryFunction = Callable[[float, float], float]
Grid = Tuple[np.ndarray, float]
BoundaryValue = Tuple[float, float, float]

In [12]:
Function = Callable[[float], float]
BinaryFunction = Callable[[float, float], float]
HeatEquationSolution = tuple[np.ndarray, np.ndarray, np.ndarray]

### Solves heat equation

uses cheme with weights $ \sigma $ <br/>
if $ \sigma = 1 $ this is an implicit scheme and template contains few points from the new layer, linar system needs to be solved <br/>
if $ \sigma = 0 $ this is an explicit scheme and template contains only one point from the new layer

In [13]:
def solve_heat_equation(
        kappa: float,
        f: BinaryFunction,
        a: float,
        T: float,
        initial_condition: Function,
        left_boundary_condition: Function,
        right_boundary_condition: Function,
        x_grid_size: int,
        t_grid_size: int,
        scheme: Literal['implicit', 'explicit']) -> HeatEquationSolution:
    x_grid, step_x = np.linspace(0, a, x_grid_size, retstep=True)
    t_grid, step_t = np.linspace(0, T, t_grid_size, retstep=True)
    
    result = np.zeros((t_grid.size, x_grid.size))
    result[0] = np.vectorize(initial_condition)(x_grid)

    coefficient =  step_t * kappa / (step_x ** 2)
    for i in range(1, t_grid.size):
        # sigma = 0
        if scheme == 'explicit':
            result[i, 0] = left_boundary_condition(t_grid[i])
            result[i, -1] = right_boundary_condition(t_grid[i])
            for j in range(1, x_grid.size - 1):
                prev_result = result[i - 1]
                result[i, j] = (prev_result[j] + 
                            coefficient * (prev_result[j - 1] - 2 * prev_result[j] + prev_result[j + 1]) + 
                            step_t * f(x_grid[j], t_grid[i - 1]))
        
        # sigma = 1 
        if scheme == 'implicit':
            matrix = lil_matrix((x_grid.size, x_grid.size))
            b = np.zeros(x_grid.size)

            for j in range(1, x_grid.size - 1):
                matrix[j, j - 1] = -coefficient
                matrix[j, j] = 2 * coefficient + 1
                matrix[j, j + 1] = -coefficient
                b[j] = step_t * f(x_grid[j], t_grid[i]) + result[i - 1][j]

            matrix[0, 0], matrix[-1, -1] = 1, 1
            b[0], b[-1] = left_boundary_condition(t_grid[i]), right_boundary_condition(t_grid[i])

            result[i] = spsolve(csc_matrix(matrix), b)

    return x_grid, t_grid, result

In [14]:
result = lambda x, t: t ** 2 * cos(x) + 0.5 * x ** 2
u_t = lambda x, t: 2 * t * cos(x)
u_xx = lambda x, t: t ** 2 * -cos(x) + 1
kappa = 2
f = lambda x, t: u_t(x, t) - kappa * u_xx(x, t)

a = 1
T = 1

initial_condition = lambda x: result(x, 0)
left_boundary_condition = lambda t: result(0, t)
right_boundary_condition = lambda t: result(a, t)


fig = make_subplots(rows=3, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}],
                           [{'is_3d': True}, {'is_3d': True}],
                           [{'is_3d': True}, {'is_3d': True}]], 
                    vertical_spacing=0.15,
                    subplot_titles=['Exact solution', 'Inplicit scheme',
                                    'Explicit scheme (2kt * 2 = h ^ 2)', 'Explicit scheme (2kt * 0.5 = h ^ 2)',
                                    'Explicit scheme (2kt * 0.978 = 1.01 h ^ 2)', 'Explicit scheme (2kt * 0.95 = h ^ 2)'])

x_grid_size = 11
t_grid_size = 11
step_x = (a - 0) / (x_grid_size - 1)

x_grid, t_grid, sol_u = solve_heat_equation(kappa, f, a, T, initial_condition, left_boundary_condition, right_boundary_condition, x_grid_size, t_grid_size, 'implicit')
fig.append_trace(go.Surface(x=x_grid, y=t_grid, z=sol_u, showscale=False), row=1, col=2)

grid = np.meshgrid(x_grid, t_grid)
exact_u = np.vectorize(result)(*grid)
fig.append_trace(go.Surface(x=x_grid, y=t_grid, z=exact_u, colorscale='tropic', showscale=False), row=1, col=1)

def draw_plot_with_stability_condition(row: int, column: int, stability_coefficitent: float):
       step_t = step_x ** 2 / kappa / (2 * stability_coefficitent)
       num_t = round((T - 0) / step_t) + 1
       x, t, solution = solve_heat_equation(kappa, f, a, T, initial_condition, left_boundary_condition, right_boundary_condition, x_grid_size, num_t, 'explicit')
       fig.append_trace(go.Surface(x=x, y=t, z=solution, showscale=False), row=row, col=column)

draw_plot_with_stability_condition(2, 1, 2)
draw_plot_with_stability_condition(2, 2, 0.5)
draw_plot_with_stability_condition(3, 1, 0.978)
draw_plot_with_stability_condition(3, 2, 0.95)

fig.update_scenes(yaxis_title='t', zaxis_title='u(x, t)')
fig.update_scenes(zaxis_tickformat='.0e', row=2, col=1)
fig.update_traces(hovertemplate='x: %{x}<br>t: %{y}<br>u(x, t): %{z}<extra></extra>')
fig.update_layout(height=900, margin=dict(r=10, l=10, b=25, t=20))

fig.show()